In [3]:
%matplotlib inline
import numpy as np
from scipy.optimize import minimize
from qiskit import *

<h1 align="center">
Simulation of a many-body quantum system
</h1>

**The problem:** Simulating many body quantum systems is a difficult task on classical computer due to exponential increase in the data with increase in number of particles. One important aspect of simulation is to calculate the ground state energy. This becomes impossible on classical systems as the number of particles grow.

**The solution:**  Using Variational quantum eigensolver(VQE) on a quantum computer Quantum computer, that finds the minimum energy of a given hamiltonian.
$\langle \psi | H | \psi \rangle \geq E_i \rightarrow$ the expectation value of a Hamiltonian will always be greater than or equal to the energy of the ground state of this system.


We will be solving for the ground state energy of a two particle system. As explained in the PDF a two particle system interaction can be described using the following Hamiltonian

$$\widehat{H}=A\left(\begin{array}{cccc}
\frac{1}{2} & 0 & 0 & 0 \\
0 & -\frac{1}{2} & 1 & 0 \\
0 & 1 & -\frac{1}{2} & 0 \\
0 & 0 & 0 & \frac{1}{2}
\end{array}\right)$$


* We use VQE to find the groud state of the Hamiltonian:$H=\begin{pmatrix}
\frac{1}{2} & 0 &0  & 0 \\
0 & -\frac{1}{2} & 1 & 0 \\
0 & 1 & -\frac{1}{2} & 0 \\
0 & 0 & 0 & \frac{1}{2}
\end{pmatrix},$
 where we have ignored the $A$ for now.


### VQE ON A **IDEAL** SIMULATOR

* First check the eigenvalues of $H$ and find the lowest one which corresponds to the lowest energy.

In [4]:
H = np.mat("0.5 0 0 0; 0 -0.5 1 0; 0 1 -0.5 0; 0 0 0 0.5") #The Hamiltonian matrix
print("The eigenvalues of H :", np.linalg.eigvals(H))
print('The ground state energy is: ', np.linalg.eigvals(H)[1])

The eigenvalues of H : [ 0.5 -1.5  0.5  0.5]
The ground state energy is:  -1.5


* As can be seen the lowest eigenvalue is $-\frac{3}{2}$.

## Variational Quantum Eingensolver (VQE):

### - Now let us find this value using VQE and see the accuracy of its result

* First of all, we need to be able to express the Hamiltonian matrix in a quantum circuit, so that the expectations values can be measured using the quantum computer.

* To achieve this, wedecompose $H$ into the sum of tensor products of Pauli operators, multiplied by some constant coefficients: $H = a \cdot II + b \cdot XX + c \cdot ZZ + d \cdot YY.$

    This equation leads to $4$ equations with $4$ variables, which can be written as $Ax=C$ :
$$
\begin{pmatrix}
1 & 0 & 1  & 0 \\
0 & 1 & 0 & -1 \\
1 & 0 & -1 & 0 \\
0 & 1 & 0 & 1
\end{pmatrix}.
\begin{pmatrix}
a \\
b \\
c \\
d
\end{pmatrix} =
\begin{pmatrix}
1/2 \\
0 \\
-1/2 \\
1
\end{pmatrix}
$$

The coefficients can be found as following

In [45]:
A = np.array([[1,0,1,0],[0,1,0,-1],[1,0,-1,0],[0,1,0,1]])
C = np.array([1/2,0,-1/2,1])

# The matrix that contains the coefficients
S = np.linalg.solve(A,C) # x = A^-1 * C
a, b, c, d = S[0], S[1], S[2], S[3]
print("a:", a,"\n b:", b, "\n c:", c, "\n d:", d)

a: 0.0 
 b: 0.5 
 c: 0.5 
 d: 0.5


Thus we have  $H = 0 \cdot II + 0.5 \cdot XX + 0.5 \cdot ZZ + 0.5 \cdot YY.$ We will convert $H$ to a dictionary for convenience later.

In [46]:
H ={'II':a, 'XX':b, 'ZZ':c,'YY':d}
print(H)

{'II': 0.0, 'XX': 0.5, 'ZZ': 0.5, 'YY': 0.5}


### - The Ansatz:

* The VQE method consists of calculating the expectation value of $H$, on various possible states in order to find that particular state which gives the minimum energy, i.e.,  $min_\theta\left\langle \psi \right| H \left| \psi \right\rangle = lowest\; energy$.

* We initialize our circuit with a guess, or ansatz: $$(I \otimes X).(C_{NOT}).(R_z(\theta) \otimes I).(H \otimes I) \left| 0 \right\rangle \otimes \left| 0 \right\rangle$$

In [36]:
# The ansatz initialization
def ansatz_init(circuit, parameter):
    q = circuit.qregs[0]
    circuit.h(q[0])
    circuit.rz(parameter, q[0])
    circuit.cx(q[0], q[1])
    circuit.x(q[1])
    return circuit

### - Transfer to the $Z\; basis$:
* The only physical measurement that a quantum computer can do, is in the Z basis {${\left| 0 \right\rangle; \left| 1 \right\rangle}$}. And as we can see, the Hamiltonian is decomposed into a linear combination of Pauli's matrices, as these form a basis for hermitian matrices. Hence, we have three kinds of measurement basis:
$$Z basis: {\left| 0 \right\rangle; \left| 1 \right\rangle},\qquad X basis: {\left| + \right\rangle; \left| - \right\rangle},\qquad Y basis: {\left| +i \right\rangle; \left| -i \right\rangle}.$$
Thus, we need to express the last two basis in the $Z\;basis$ :

 $X\;basis$ in term of $Z\;basis$; we get the matrix: $H^{'} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix}$.

 $Y\;basis$ in term of $Z\;basis$; we get the matrix: $Y^{'} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
i & -i
\end{pmatrix}$.

Those two matrices transfer from {${\left| 0 \right\rangle; \left| 1 \right\rangle}$} to {${\left| + \right\rangle; \left| - \right\rangle}$} and {${\left| +i \right\rangle; \left| -i \right\rangle}$} respectevly.
    
Every quantum state $\left| \psi \right\rangle$ can be expressed in different sets of basis, so if we want it to be written in the {$\left| 0 \right\rangle; \left| 1 \right\rangle$} basis, to get the count of the measurement to calculate the expectations values, we process as follow:

 * We apply the inverse of $H^{'}$ to go from the {${\left| + \right\rangle; \left| - \right\rangle}$} basis to the {${\left| 0 \right\rangle; \left| 1 \right\rangle}$} basis; consequently, we get the matrix: $H_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix}$.
 * We apply the inverse of $Y^{'}$ to go from the {${\left| +i \right\rangle; \left| -i \right\rangle}$} basis to the {${\left| 0 \right\rangle; \left| 1 \right\rangle}$} basis; then, we get the matrix: $Y_{gate} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -i\\
1 & i
\end{pmatrix}$.

In [37]:
# transfer to the Z basis measurement circuit
def z_measure_circ(parameter, measure):
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    circuit = QuantumCircuit(q, c)

    # implement the ansatz in the circuit
    circuit = ansatz_init(circuit, parameter)

    # measurement
    if measure == 'XX':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi, q[0])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'ZZ':
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'YY':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi/2, q[0])
        circuit.u(np.pi/2,0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    else:
         raise ValueError('Input should be "XX" or "YY" or "ZZ"')

    return circuit

* The counts result is returned in qiskit, in a form of a dictionary, so we will define a function ```get_from``` that returns the values in the dictionary.

In [38]:
# If key is missing than return 0 otherwise the corresponding value.
def get_from(d: dict, key: str):

    value = 0
    if key in d:
        value = d[key]
    return value

### - The expectation value:

* The expectation value in the case of two qubits, is calculated considering the computational basis (Z-Axis) as follow: eigenvalues of the Pauli's $Z$ matrix times the corresponding probabilities
$$\frac{1}{N}.(N_{00}\left\langle 00 \right| Z \otimes Z \left| 00 \right\rangle + N_{11}\left\langle 11 \right| Z \otimes Z \left| 11 \right\rangle + N_{10}\left\langle 10 \right| Z \otimes Z \left| 10 \right\rangle + N_{01}\left\langle 01 \right| Z \otimes Z \left| 01 \right\rangle)$$
In which:
$$\left\langle 00 \right| Z \otimes Z \left| 00 \right\rangle = \left\langle 11 \right| Z \otimes Z \left| 11 \right\rangle = 1$$
and
$$\left\langle 10 \right| Z \otimes Z \left| 10 \right\rangle = \left\langle 01 \right| Z \otimes Z \left| 01 \right\rangle = -1$$
therefore the expectation value become:
$$\frac{1}{N}.((N_{00} + N_{11}) - (N_{10} + N_{01}))$$
where $N$ is the total number of shots, and $N_{00}$, $N_{11}$, $N_{10}$, $N_{01}$ are the number of time the state $\left| 00 \right\rangle$, $\left| 11 \right\rangle$, $\left| 10 \right\rangle$, $\left| 01 \right\rangle$ are measured respectively.

In [39]:
#calculate the expectation value for each Pauli's gate
def expec_value(parameter, measure):
    # measure in the right basis, then use the counts to compute the expectation value.
    if measure == 'II':
        return 1
    if measure == 'XX':
        circuit = z_measure_circ(parameter, 'XX')
    elif measure == 'ZZ':
        circuit = z_measure_circ(parameter, 'ZZ')
    elif measure == 'YY':
        circuit = z_measure_circ(parameter, 'YY')
    else:
        raise ValueError('Input should be "II" or "XX" or "ZZ" or "YY"')

    shots = 1000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()

    expectation_value = ((get_from(counts, '00')+get_from(counts, '11')) -
                         (get_from(counts,'10')+get_from(counts, '01'))) / shots

    return expectation_value

* We will now create a function ```sum_expec``` that sums up all the expectation value inside the one of the Hamiltonian, multiply them by their coefficients, and returns the final result.
$$\left\langle H \right\rangle = a \cdot \left\langle \psi \right| II \left| \psi \right\rangle + b \cdot \left\langle \psi \right| XX \left| \psi \right\rangle + c \cdot \left\langle \psi \right| ZZ \left| \psi \right\rangle + d \cdot \left\langle \psi \right| YY \left| \psi \right\rangle.$$

In [40]:
def sum_expec(parameter):
    if isinstance(parameter, np.ndarray):
        parameter = parameter[0]
    expec_value_II = get_from(H, 'II') * expec_value(parameter, 'II') #a*<II>
    expec_value_XX = get_from(H, 'XX') * expec_value(parameter, 'XX') #b*<XX>
    expec_value_ZZ = get_from(H, 'ZZ') * expec_value(parameter, 'ZZ') #c*<ZZ>
    expec_value_YY = get_from(H, 'YY') * expec_value(parameter, 'YY') #d*<YY>
    # summing the expectations results
    sum_result = expec_value_II + expec_value_XX + expec_value_ZZ + expec_value_YY

    return sum_result

### Using the optimizer:
* the ansatz we had initialized has only one degree of freedom i.e. it depends on one parameter $\theta$ (rotation) on which we can vary, so the trial wavefunctions depend on $\theta$; $\left| \psi(\theta) \right\rangle$.

<b> So, we will use optimizer called ```minimize``` from ```scipy.optimize```, to search the best angle $\theta$, therefore the best wavefunction $\left| \psi(\theta) \right\rangle$, which will minimize the expectation value: $min_\theta\left\langle \psi(\theta) \right| H \left| \psi(\theta) \right\rangle = lowest\; energy\;$.

In [48]:
from scipy.optimize import minimize_scalar
parameter = 1 #initialize an arbitrary angle

tol = 1e-3 #tolerance for optimization precision.
#Inject the sum_expec result inside the minimizer function
sum_expec_result = minimize(sum_expec, parameter, method="Powell", tol=tol)

print('The exact ground state energy is: {}'.format(-1.5))
print('The estimated ground state energy using VQE algorithm is: {}'.format(sum_expec_result.fun))
print("\nThe optimal parameter theta is : {} ".format(sum_expec_result.x))

The exact ground state energy is: -1.5
The estimated ground state energy using VQE algorithm is: -1.5

The optimal parameter theta is : [3.14387264] 


* As can be seen, the optimizer can ﬁnd the best angle $\theta = 3.1454328877068733 \approx \pi \;$ for the global minimum energy $ E = \left\langle \psi \right| H \left| \psi \right\rangle = -1.5$ (in arbitrary units) as expected.

### Final results:

**Optimizing over the expectation values for all angles, lead to the optimal angle $\theta=\pi$, therefore gives the lowest eigenstate $|\psi(\theta)\rangle$ (the right ansatz), as follow:**

In [16]:
######## Ansatz Circuit ########
qc = QuantumCircuit(2)
qc.h(0)
qc.rz(np.pi,0)
qc.cx(0,1)
qc.x(1)
qc.draw()

┌───┐┌───────┐          
q_0: ┤ H ├┤ Rz(π) ├──■───────
     └───┘└───────┘┌─┴─┐┌───┐
q_1: ──────────────┤ X ├┤ X ├
                   └───┘└───┘

In [21]:
from qiskit_textbook.tools import array_to_latex
######## Ansatz State-vector ########
backend2 = Aer.get_backend('statevector_simulator')
final_state = execute(qc,backend2).result().get_statevector()
array_to_latex(final_state, pretext=r"\\Statevector: \; |\psi(\theta)\rangle = ")

<IPython.core.display.Math object>

**The corresponding lowest energy is $\left\langle \psi(\pi)| H |\psi(\pi)\right\rangle = -\frac{3}{2}A$.**
